# Projeto 1 - Ciência dos Dados

Nome: Fernando Bichuette Assumpção

Nome: Felipe Lacombe 

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [52]:
%matplotlib inline
import matplotlib.pyplot as plt
import tweepy
import math
import os.path
import pandas as pd
import json
import re
import string
import emoji
import functools
import operator
from time import sleep
from random import shuffle

In [53]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Fernando\Documents\Insper\3° Semestre\Ciencia dos Dados\C_Dados_Projeto1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [54]:
#importando database
dados=pd.read_excel("bolsa.xlsx",'Treinamento')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

No projeto 1 de Ciência de dados será feito um clasificador que vai analizar a palavra bolsa em todos os tweets e classificar se é relevante quando está relacionada a **bolsa de valores** e irrelevante quando é empregada em outro em outro sentido ex: bolsa familia, bolsa utilitaria e etc.


## Limpando database

In [55]:
dados.B1.value_counts()

1    153
0    147
Name: B1, dtype: int64

In [90]:
#fazendo a limpeza do arquivo
def limpar_caracteres(tabela, titulo):
    i = tabela[titulo]
    i = i.str.lower()

    itens = ['.', ':', ';', '"', "'", '?', '(', ')', '[',']',',', '\n', '\t','@','?','%',]
    i2 = ['?', '!']
    emojis = emoji_list.all_emoji
    for emoji in emojis:
        try:
            i = i.str.replace(emoji," {} ".format(emoji) )
        except:
            pass
    for e in itens:
        i = i.str.replace(e,'')
    for e in i2:
        i = i.str.replace(e,' {} '.format(e))

    i = i.str.replace('é','e')
    i = i.str.replace('ê','e')
    i = i.str.replace('á','a')
    i = i.str.replace('ã','a')
    i = i.str.replace('ô','o')
    i = i.str.replace('ó','o')
    i = i.str.replace('ú','u')
    i = i.str.replace('ç','c')
    i = i.str.replace('í','i')
    i = i.str.replace('@','')
    i = i.str.replace('?','')
    i = i.str.replace(',','.')
        
    tabela[titulo] = i

    return tabela


titulo="Treinamento"

tabela=limpar_caracteres(tabela, titulo)

def separador(tabela, titulo):
    i = tabela[titulo].str.split(' ')

    nova = pd.DataFrame()
    nova['Tabela splitted'] = []

    for lista in i:
        for e in ['#','' , ' ', 'rt']:
            while lista.count(e) != 0:
                lista.remove(e)

        for palavra in lista:
            
            if palavra[:4] == 'http':
                lista.remove(palavra)
        nova = nova.append({'Tabela splitted': lista}, ignore_index=True)
                
    return nova
nova=separador(dados,titulo)


def tabela_palavras(tabela):
    a = tabela['Tabela splitted']
    prob = pd.DataFrame()
    prob['Tabela splitted'] = []
    for lista in a:
        for e in lista:
            prob = prob.append({'Tabela splitted': e}, ignore_index=True)
    
    return prob['Tabela splitted']
dados_palavras=tabela_palavras(nova)
nova.head()


,Tabela splitted
0,"[dólar, cai, a, r$, 4,64,, e, bolsa, avança, m..."
1,"[@albuquerque_af, @tradersclubbr, entrei, no, ..."
2,"[gente, eu, amo, esses, perfuminhos, da, jequi..."
3,"[minha, colega, pediu, um, chiclete,, fui, pro..."
4,"[bolsa, cai,, dólar, sobe,, o, país, se, desma..."


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [57]:
palavra=dados_palavras.value_counts(True) #probabilidade de sorteio das palavras que possuem a maior ocorrencia no dataframe
palavra.head()

bolsa    0.038183
a        0.031255
de       0.030947
e        0.028791
o        0.021093
Name: Tabela splitted, dtype: float64

In [58]:
#Dataframe relevante 
relevantes = dados.loc[(dados['B1'])==1]
relevantes.set_index('B1',inplace=True)
relevantes.head()

,Treinamento
B1,
1,"dólar cai a r$ 4,64, e bolsa avança mais de 7%..."
1,@albuquerque_af @tradersclubbr entrei no final...
1,"bolsa cai, dólar sobe, o país se desmancha (9...."
1,@brunosouzasc você pode me explicar onde está ...
1,📈🇧🇷 #últimahora la bolsa de sao paulo se dispa...


In [74]:
#Dataframe irrelevante
irrelevantes = dados.loc[(dados['B1'])==0]
irrelevantes.set_index('B1',inplace=True)
irrelevantes.head()

,Treinamento
B1,
0,gente eu amo esses perfuminhos da jequiti d le...
0,"minha colega pediu um chiclete, fui procurar n..."
0,em 2020 estamos com uma fila enorme no bolsa f...
0,@lulaoficial cuidar dos pobres é uma maravilha...
0,uma das minhas maiores certezas na vida era qu...


In [92]:
#Split os tweets dos dataframes criados acima
relevante_separado=separador(relevantes,titulo)
irrelevante_separado=separador(irrelevantes,titulo)

In [81]:
#calculando quantas vezes cada palavra aparece nos tweets relevantes
splited_relevante=tabela_palavras(relevante_separado)
prob_relevante = splited_relevante.value_counts(True) 
num_palavras_relevante = splited_relevante.value_counts()
num_palavras_relevante.head()

bolsa    125
a        119
de       110
e         98
o         81
Name: Tabela splitted, dtype: int64

In [83]:
#calculando quantas vezes cada palavra aparece nos tweets irrelevantes
splited_irrelevante=tabela_palavras(irrelevante_separado)
prob_irrelevante = splited_irrelevante.value_counts(True) 
num_palavras_irrelevante = splited_irrelevante.value_counts()
num_palavras_irrelevante.head()

bolsa    123
de        91
e         89
a         84
o         56
Name: Tabela splitted, dtype: int64

In [78]:
# probabilidade de ocorrencia da palavra bolsa dentro do database relevante
prob_relevante['bolsa']

0.03653902367728734

In [85]:
#calculando a quantidade total de palavras temos em nosso database
tamanho=[]
newnova=nova.iloc[0:,0]
for t in newnova:
    for b in t:
        tamanho.append(b)

In [80]:
#criando novas colunas para o database final
nova["Classificacao Manual"] = pd.Series(dados["B1"])
nova["Classificacao_Naive"] = pd.Series()
nova["Resposta"] = pd.Series()

### Suavização de Laplace

In [87]:
#criando um contador para percorrer o database linha por linha
a=0
#criando as variaveis que serão usadas na suavização de Laplace
total=len(tamanho)
total_relevantes=len(num_palavras_relevante)
total_irrelevantes=len(num_palavras_irrelevante)
len_01=len(num_palavras_relevante)
len_00=len(num_palavras_irrelevante)
#fazendo a suavização de Laplace
for valor in nova["Tabela splitted"]:
    alpha = 1
    prob01 = 1
    prob00 = 1
    for val in valor:
        #caso a palavra não exista no nosso database, terá seu valor alterado, para uma melhor 
        #classificação do Naive Bayers
        if val not in num_palavras_relevante or val not in num_palavras_irrelevante:
            sec01 = alpha
            sec00 = alpha
        
        else:
            sec00=num_palavras_irrelevante[val] + alpha
            sec01=num_palavras_relevante[val] + alpha
            
            
        if val in num_palavras_relevante:
            prob01 *= (num_palavras_relevante[val] + alpha) / (total_relevantes + total*alpha)
        else: 
            prob01 *= sec01/(len(num_palavras_relevante)+total*alpha)
        
        if val in num_palavras_irrelevante:   
            prob00 *= (num_palavras_irrelevante[val] + alpha) / (total_irrelevantes + total*alpha)
        else:
            prob00 *= sec00/(len(num_palavras_irrelevante)+total*alpha)
       
    if prob01>prob00:
        prob_x = 1
        nova["Classificacao_Naive"][a] = prob_x
        if nova["Classificacao Manual"][a] == prob_x:
            nova["Resposta"][a] = "Positivo = True"
        elif nova["Classificacao Manual"][a] != prob_x:
            nova["Resposta"][a] = "Positivo = False"
            
    elif prob01<prob00:
        prob_x = 0
        nova["Classificacao_Naive"][a] = prob_x
        if nova["Classificacao Manual"][a] == prob_x:
            nova["Resposta"][a] ="Negativo = True"
        elif nova["Classificacao Manual"][a] != prob_x:
            nova["Resposta"][a] ="Negativo = False"
    a+=1
resultado=nova

C:\Users\Fernando\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Fernando\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Fernando\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Fernando\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**